In [ ]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.insert(0, '..')
import ccal

%matplotlib inline
# %config InlineBackend.figure_formats = {'svg',}

# Get states

In [ ]:
h = ccal.support.read_gct('/home/cyborg/hematopoietic_cancer/result/ccle_hema_r/k9/ccle_hema_k_9_H.gct')
n_states_to_explore = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30]
nclustering = 500
filename_prefix = 'output/ccle_hema_k_9_H_1000clustering_0828'

In [ ]:
np.argmax(np.asarray(h), axis=0)

In [ ]:
# labels, memberships, cophenetic_correlation = ccal.analyze.get_states_from_h(h, n_states_to_explore, nclustering=nclustering, filename_prefix=filename_prefix)

# Map Onco-GPS

In [ ]:
labels = pd.read_csv(filename_prefix + '_labels.txt', index_col=0, sep='\t')
memberships = ccal.support.read_gct(filename_prefix + '_memberships.gct')
annotations = ccal.support.make_random_features(1, memberships.shape[1])

ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]))#,
#                              background_markersize=0, background_mask_markersize=0)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]))#,
#                              background_markersize=0, background_mask_markersize=0, mds_is_metric=False)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]))#,
#                              background_markersize=0, background_mask_markersize=0, informational_mds=False)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]),
                             annotations=np.random.random_sample(h.shape[1]))#
#                              background_markersize=0, background_mask_markersize=0)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]),
                             annotations=np.random.choice(range(5), h.shape[1]),
                             annotation_type='categorical')#
#                              background_markersize=0, background_mask_markersize=0)
ccal.visualize.plot_onco_gps(h,
                             np.random.choice(range(10), h.shape[1]),
                             annotations=np.random.choice(range(2), h.shape[1]),
                             annotation_type='binary')#
#                              background_markersize=0, background_mask_markersize=0)

In [ ]:
normalize_pandas_object = ccal.support.normalize_pandas_object
compare_matrices = ccal.support.compare_matrices
f = ccal.information.information_coefficient

In [ ]:
normalized_clipped_h = normalize_pandas_object(normalize_pandas_object(h).clip(-3, 3), method='0-1')

X = compare_matrices(normalized_clipped_h, normalized_clipped_h,
                 f, report_progress=False)
X

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15, 15))
sns.heatmap(X, annot=True)

In [ ]:
a = np.random.random_sample(100)
b = np.random.random_sample(100)
print(f(a, a))
print(f(b, b))
print(f(a, b))

In [ ]:
for n_state in [12]:# n_states_to_explore:
    states = labels.ix[n_state, :-1].astype(int).tolist()
    signature = 'mds_informational_metric_2'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=True, informational_mds=True, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature),
                                mds_seed=2016)
    signature = 'mds_informational_no_metric_2'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=False, informational_mds=True, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature),
                                mds_seed=2016)
    signature = 'mds_no_informational_metric_2'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=True, informational_mds=False, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature),
                                mds_seed=2016)
    signature = 'mds_no_informational_no_metric_2'
    ccal.visualize.plot_onco_gps(h, states, mds_is_metric=False, informational_mds=False, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature),
                                mds_seed=2016)
    signature = 'populate_random'
#     ccal.visualize.plot_onco_gps(h, states, annotations=annotations, output_filepath='output/ccle_hema_k_9_H_1000clustering_0828_s{}_{}.pdf'.format(n_state, signature))